<a href="https://colab.research.google.com/github/juglar-diaz/STTD/blob/master/Represent_As_Doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Intro

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(accelerator)
!pip3 install gensim


cpu


In [4]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import itertools
from gensim import models
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from scipy.sparse import csr_matrix

import random



import pickle

import os
sep = os.sep
import os.path

import pandas as pd

import time


if(os.path.isfile('utils.py')):
    !rm 'utils.py'
    print("deleted")

exchangedrive = drive.CreateFile({'id':'12djgLg96tD7ynEhImcqoYnL5ZamenDEM'})
exchangedrive.GetContentFile('utils.py')

from utils import *


deleted


#Download Data

In [0]:
exchangedrive = drive.CreateFile({'id':'1cMCzlvTMlUPgaYaUIdlMVtTpp8r10GnX'})
exchangedrive.GetContentFile('robosclean.p')


exchangedrive = drive.CreateFile({'id':'1A5Wa6LiaGs8XeW2S2qjGbcoSSlsMGW97'})
exchangedrive.GetContentFile('tweetsLAtrain.csv')
exchangedrive = drive.CreateFile({'id':'1CrUCS7oWzdvYoFwtWgikiGkDAu6w3dOF'})
exchangedrive.GetContentFile('tweetsLAtest.csv')
exchangedrive = drive.CreateFile({'id':'1AU_saqdyRxtgMX2TC7Q42OBIUm68cdw_'})
exchangedrive.GetContentFile('tweetsLA.csv')



exchangedrive = drive.CreateFile({'id':'1NuSVM7-h0CCRtzi0woM4h5tVd6T7JO2C'})
exchangedrive.GetContentFile('tweetsNYtrain.csv')
exchangedrive = drive.CreateFile({'id':'1UYZY0sh1-Q8MIofMAHDGaNKv8cfDA0ui'})
exchangedrive.GetContentFile('tweetsNYtest.csv')


exchangedrive = drive.CreateFile({'id':'1Jv19eJTZwsZWEA_rUudvKwcn1TS-DFBa'})
exchangedrive.GetContentFile('tweets2016_2half.csv')
exchangedrive = drive.CreateFile({'id':'1E8WlhOXb3tfQbLUpizx7QXzeydv57eXN'})
exchangedrive.GetContentFile('toy_2017_Jan.csv')



#Models

In [0]:
class RepreAsDoc():
    def __init__(self):
        pass


    def fit(self, tuples_train, tf_idf = False, lda = False):#t is a tuple (date_out, coor_out, tweets_out)

        ldates_as_doc = []# a list of [wordidx,...,wordidx]), where the index of the list is a dateidx or a cooridx or a wordidx
        lcoor_as_doc = []# a list of [wordidx,...,wordidx]), where the index of the list is a cooridx

        self.max_idxdate = max([tuple[0] for tuple in tuples_train])
        #print(self.max_idxdate)
        self.min_idxword = max([tuple[1] for tuple in tuples_train]) + 1
        #print(self.min_idxword)


        max_idxword = 0
        for tuple in tuples_train:
            for idxword in tuple[2]:
                if idxword > max_idxword:
                    max_idxword = idxword
        #print(max_idxword)
        self.max_idxword = max_idxword


        min_idxword = max_idxword
        for tuple in tuples_train:
            for idxword in tuple[2]:
                if idxword < min_idxword:
                    min_idxword = idxword
        #print(min_idxword)

        vectors = [[] for _ in range(max_idxword+1)]

        for i in range(max_idxword+1):
            vectors[i] = [0 for _ in range(max_idxword-min_idxword + 1)]

        for tuple in tuples_train:
            dateidx, cooridx, wordsidxs = tuple[0], tuple[1], tuple[2]
            for idxword in wordsidxs:
                vectors[dateidx][idxword-min_idxword] += 1
                vectors[cooridx][idxword-min_idxword] += 1

                for word_idx in wordsidxs:
                    vectors[word_idx][idxword-min_idxword] += 1

        if(tf_idf and not lda):
            tfidf_model = TfidfTransformer()
            vectors = tfidf_model.fit_transform(vectors)
            vectors = vectors.todense()

        if(tf_idf and lda):
            tfidf_model = TfidfTransformer()
            vectors = tfidf_model.fit_transform(vectors)

            lda_model = LatentDirichletAllocation(n_components=20, max_iter=5,
                                            learning_method='online',
                                            learning_offset=50.,
                                            random_state=0)
            vectors = lda_model.fit_transform(vectors)

        if (not tf_idf and lda):

            vectors = csr_matrix(vectors)

            lda_model = LatentDirichletAllocation(n_components=20, max_iter=5,
                                                  learning_method='online',
                                                  learning_offset=50.,
                                                  random_state=0)
            vectors = lda_model.fit_transform(vectors)

        self.W = np.array(vectors)

    def predict(self, t_idx, l_idx, w_idx):
        vec_t = self.W[t_idx].reshape(1, -1)
        vec_l = self.W[l_idx].reshape(1, -1)

        vec_w = np.average([self.W[idx].reshape(1, -1) for idx in w_idx], axis=0)

        vectors = [vec_t, vec_l, vec_w]
        #print(vec_t.shape, vec_l.shape, vec_w.shape)
        #for vec1, vec2 in itertools.combinations(vectors, r=2):
        #    if (cosine_similarity(vec1, vec2).shape != (1, 1)):
        #        print('cosine_similarity shape error')
        #    print(cosine_similarity(vec1, vec2))
        score = sum([cosine_similarity(vec1, vec2)[0][0] for vec1, vec2 in itertools.combinations(vectors, r=2)])
        return round(score, 6)

    def getPlaces(self):
        return self.W[self.max_idxdate+1:self.min_idxword,:]

    def queryItem(self, idxs, number= 10):
        query_vectors = self.W[idxs]
        query_vectors.reshape(query_vectors.shape[0],1,-1)
        #print(self.max_idxdate+1)

        date_vectors = self.W[list(range(0,self.max_idxdate+1))]
        #print(date_vectors)

        date_vectors.reshape(date_vectors.shape[0], 1, -1)

        cosines_sim_date = cosine_similarity(query_vectors, date_vectors)



        #coor_vectors = self.W[list(range(self.max_idxdate+1,self.min_idxword))]
        coor_vectors = self.W[self.max_idxdate+1:self.min_idxword,:]
        #print(coor_vectors)

        coor_vectors.reshape(coor_vectors.shape[0], 1, -1)
        cosines_sim_coor = cosine_similarity(query_vectors, coor_vectors)

        word_vectors = self.W[list(range(self.min_idxword, self.max_idxword + 1))]
        word_vectors.reshape(word_vectors.shape[0], 1, -1)
        cosines_sim_word = cosine_similarity(query_vectors, word_vectors)


        indexes_date = np.argsort(cosines_sim_date, )
        indexes_coor = np.argsort(cosines_sim_coor) + self.max_idxdate+1
        indexes_word = np.argsort(cosines_sim_word) + self.min_idxword

        return (indexes_date[:1,-number:], indexes_coor[:1,-number:], indexes_word[:1,-number:])

    def queryVector(self, query_vectors, number= 10):
        query_vectors = np.array(query_vectors)
        query_vectors.reshape(query_vectors.shape[0],1,-1)
        #print(self.max_idxdate+1)

        date_vectors = self.W[list(range(0,self.max_idxdate+1))]
        #print(date_vectors)

        date_vectors.reshape(date_vectors.shape[0], 1, -1)

        cosines_sim_date = cosine_similarity(query_vectors, date_vectors)



        #coor_vectors = self.W[list(range(self.max_idxdate+1,self.min_idxword))]
        coor_vectors = self.W[self.max_idxdate+1:self.min_idxword,:]
        #print(coor_vectors)

        coor_vectors.reshape(coor_vectors.shape[0], 1, -1)
        cosines_sim_coor = cosine_similarity(query_vectors, coor_vectors)

        word_vectors = self.W[list(range(self.min_idxword, self.max_idxword + 1))]
        word_vectors.reshape(word_vectors.shape[0], 1, -1)
        cosines_sim_word = cosine_similarity(query_vectors, word_vectors)


        indexes_date = np.argsort(cosines_sim_date, )
        indexes_coor = np.argsort(cosines_sim_coor) + self.max_idxdate+1
        indexes_word = np.argsort(cosines_sim_word) + self.min_idxword

        return (indexes_date[:1,-number:], indexes_coor[:1,-number:], indexes_word[:1,-number:])


#Eval

In [8]:
%%time
translate = {'w':'Text','l':'Location','t':'Time'}

namesets = ['Santiago','LA']
results = {'Model':[],'Dataset':[],  'Text':[], 'Location':[],'Time':[]}



for i in range(1):
    trainST,word2idxST,testST = chargeData_SplitTrainTest('tweets2016_2half.csv')
    trainLA,word2idxLA,testLA = chargeData_SplitTrainTest("tweetsLA.csv")
    
    trainsets = [trainST,  trainLA]
    testsets = [testST,  testLA]
    for train,test,name in list(zip(trainsets, testsets, namesets)):
        QE = QuantitativeEvaluator()


        model = RepreAsDoc()

        ldadict={True:'lda', False:''}
        tfidfdict={True:'tf_idf', False:''}

        for lda in [False, True]:
            for tfidf in [False, True]:
                print(lda,tfidf)
                model.fit(train, lda=lda, tf_idf=tfidf)

                print('Done Training')


                for predict_type in 'wlt':
                    QE.get_ranks(test, model, predict_type = predict_type)
                    mrr1 = QE.compute_mrr()[0]
                    print(f"{name} MRR {predict_type} {mrr1}")
                    results[translate[predict_type]].append(mrr1)
                results['Dataset'].append(name)
                results['Model'].append(str(type(model))+'_'+tfidfdict[tfidf]+'_'+ldadict[lda])
                df = pd.DataFrame(results)

                df.to_csv('resultsRobos.df')
    
    
    
    

False False
Done Training
Santiago MRR w 0.6049
Santiago MRR l 0.446
Santiago MRR t 0.3959
False True
Done Training
Santiago MRR w 0.449
Santiago MRR l 0.4356
Santiago MRR t 0.4147
True False
Done Training
Santiago MRR w 0.4261
Santiago MRR l 0.4276
Santiago MRR t 0.4128
True True
Done Training
Santiago MRR w 0.4042
Santiago MRR l 0.3933
Santiago MRR t 0.3828
False False
Done Training
LA MRR w 0.4468
LA MRR l 0.4316
LA MRR t 0.3904
False True
Done Training
LA MRR w 0.4123
LA MRR l 0.42
LA MRR t 0.4021
True False
Done Training
LA MRR w 0.4076
LA MRR l 0.4077
LA MRR t 0.3967
True True
Done Training
LA MRR w 0.3847
LA MRR l 0.3752
LA MRR t 0.3662
CPU times: user 5h 42min 53s, sys: 4min 34s, total: 5h 47min 27s
Wall time: 5h 41min 57s


In [0]:
endrun

In [0]:
%%time
translate = {'w':'Text','l':'Location','t':'Time'}

namesets = ['Santiago','LA']
results = {'Model':[],'Dataset':[],  'Text':[], 'Location':[],'Time':[],'Batch':[],'Dim':[]}



for i in range(1):
    trainST,word2idxST,testST = chargeData_SplitTrainTest('tweets2016_2half.csv')
    trainLA,word2idxLA,testLA = chargeData_SplitTrainTest("tweetsLA.csv")
    
    trainsets = [trainST,  trainLA]
    testsets = [testST,  testLA]
    for train,test,name in list(zip(trainsets, testsets, namesets)):

        dims = [20,50,100,200,400]
        batchs = [200]


        for dim in dims:
            for batch in batchs:
                model = SimpleRNN()
                model.fit_batch(train, embedding_dims=dim, num_epochs=5, hidden_size=dim, batch_size=batch)
                print(f"Test size {len(test)}")
                for predict_type in 'wlt':
                    resultsp, results_targets = model.predict(test, batch_test=10000, predict_type=predict_type, fake = 10)
                    mrr1 = model.mrr(resultsp, results_targets, predict_type=predict_type)[0]
                    print(f"{name} MRR {predict_type} {mrr1} {dim} {batch}")
                    results[translate[predict_type]].append(mrr1)

                results['Dataset'].append(name)
                results['Dim'].append(dim)
                results['Batch'].append(batch)

                results['Model'].append(str(type(model))+'_'+str())

                df = pd.DataFrame(results)

                df.to_csv('results.df')

In [0]:
y

In [0]:
%%time
translate = {'w':'Text','l':'Location','t':'Time'}

namesets = ['Santiago','LA']
results = {'Model':[],'Dataset':[],  'Text':[], 'Location':[],'Time':[],'Batch':[],'Dim':[]}



for i in range(5):
    trainST,word2idxST,testST = chargeData_SplitTrainTest('tweets2016_2half.csv')
    trainLA,word2idxLA,testLA = chargeData_SplitTrainTest("tweetsLA.csv")
    
    trainsets = [trainST,  trainLA]
    testsets = [testST,  testLA]
    for train,test,name in list(zip(trainsets, testsets, namesets)):

        dims = [100]
        batchs = [200]


        for dim in dims:
            for batch in batchs:
                model = SimpleRNN()
                model.fit_batch(train, embedding_dims=dim, num_epochs=5, hidden_size=dim, batch_size=batch)
                print(f"Test size {len(test)}")
                for predict_type in 'wlt':
                    resultsp, results_targets = model.predict(test, batch_test=10000, predict_type=predict_type, fake = 10)
                    mrr1 = model.mrr(resultsp, results_targets, predict_type=predict_type)[0]
                    print(f"{name} MRR {predict_type} {mrr1} {dim} {batch}")
                    results[translate[predict_type]].append(mrr1)

                results['Dataset'].append(name)
                results['Dim'].append(dim)
                results['Batch'].append(batch)

                results['Model'].append(str(type(model))+'_'+str())

                df = pd.DataFrame(results)

                df.to_csv('results.df')

In [0]:
%%time
translate = {'w':'Text','l':'Location','t':'Time'}
trainsets = [trainST, trainNY, trainLA]
testsets = [testST, testNY, testLA]
namesets = ['Santiago','NY','LA']
results = {'Model':[],'Dataset':[],  'Text':[], 'Location':[],'Time':[],'Batch':[],'Dim':[]}

for train,test,name in list(zip(trainsets, testsets, namesets)):
    
    dims = [100]
    batchs = [200]
    
    
    for dim in dims:
        for batch in batchs:
            model = SimpleRNN()
            model.fit_batch(train, embedding_dims=dim, num_epochs=10, hidden_size=dim, batch_size=batch)
            print(f"Test size {len(test)}")
            for predict_type in 'ltw':
                resultsp, results_targets = model.predict(test, batch_test=10000, predict_type=predict_type, fake = 10)
                mrr1 = model.mrr(resultsp, results_targets, predict_type=predict_type)[0]
                print(f"{name} MRR {predict_type} {mrr1} {dim} {batch}")
                results[translate[predict_type]].append(mrr1)

            results['Dataset'].append(name)
            results['Dim'].append(dim)
            results['Batch'].append(batch)

            results['Model'].append(str(type(model))+'_'+str())

            df = pd.DataFrame(results)

            df.to_csv('results.df')

In [0]:
df.head(20)